### Compute Distances Between Leaves

https://rosalind.info/problems/ba7a

In [1]:
from collections import defaultdict, deque

def leaves_distance_matrix(n, adjacency_list):
    """
    n: number of leaves (indexed 0..n-1)
    adjacency_list: dict of {node: list of (neighbor, weight)}
    Returns: n x n matrix of distances between leaves
    """
    # 叶节点列表
    leaves = list(range(n))
    dist_matrix = [[0]*n for _ in range(n)]
    
    def bfs(start):
        """Return distances from start to all nodes in the graph"""
        visited = {}
        queue = deque([(start, 0)])
        while queue:
            node, d = queue.popleft()
            if node in visited:
                continue
            visited[node] = d
            for neighbor, w in adjacency_list[node]:
                if neighbor not in visited:
                    queue.append((neighbor, d + w))
        return visited
    
    # 对每个叶节点计算到所有叶节点的距离
    for i, leaf in enumerate(leaves):
        distances = bfs(leaf)
        for j, other_leaf in enumerate(leaves):
            dist_matrix[i][j] = distances[other_leaf]
    
    return dist_matrix


In [3]:
# 读取文件
with open("data/rosalind_ba7a.txt", "r") as f:
    lines = [line.strip() for line in f if line.strip()]

n = int(lines[0])
adjacency_list = defaultdict(list)

# 解析邻接表
for line in lines[1:]:
    u_v, w = line.split(':')
    u, v = map(int, u_v.split('->'))
    w = int(w)
    adjacency_list[u].append((v, w))

# 计算叶节点距离矩阵
matrix = leaves_distance_matrix(n, adjacency_list)

# 输出结果
for row in matrix:
    print('  '.join(map(str, row)))


0  95  70  81  37  153  129  192  41  119  130  107  113  55  62  62  101  82  135  200  39  84  45  175  188  37  51  164  106  129  147  84
95  0  51  152  108  224  48  263  64  190  49  178  32  126  61  133  172  41  206  271  80  155  116  246  259  88  122  235  25  200  218  33
70  51  0  127  83  199  85  238  39  165  86  153  69  101  36  108  147  38  181  246  55  130  91  221  234  63  97  210  62  175  193  40
81  152  127  0  66  100  186  139  98  66  187  54  170  36  119  31  48  139  82  147  96  31  62  122  135  94  52  111  163  76  94  141
37  108  83  66  0  138  142  177  54  104  143  92  126  40  75  47  86  95  120  185  52  69  30  160  173  50  36  149  119  114  132  97
153  224  199  100  138  0  258  55  170  58  259  64  242  108  191  103  82  211  38  63  168  89  134  38  51  166  124  27  235  50  28  213
129  48  85  186  142  258  0  297  98  224  15  212  28  160  95  167  206  75  240  305  114  189  150  280  293  122  156  269  41  234  252 

### Compute Limb Lengths in a Tree

https://rosalind.info/problems/ba7b

In [4]:
def limb_length(D, j):
    """
    D: n x n additive distance matrix (list of lists)
    j: leaf index (0-based)
    Returns: limb length of leaf j
    """
    n = len(D)
    limb = float('inf')
    for i in range(n):
        if i == j:
            continue
        for k in range(n):
            if k == j or k == i:
                continue
            val = (D[i][j] + D[j][k] - D[i][k]) / 2
            if val < limb:
                limb = val
    return int(limb)  # 返回整数


In [6]:
# 读取文件
with open("data/rosalind_ba7b.txt", "r") as f:
    lines = [line.strip() for line in f if line.strip()]

n = int(lines[0])
j = int(lines[1])
D = [list(map(int, line.split())) for line in lines[2:]]

# 计算 limb length
result = limb_length(D, j)
print(result)


948


### Implement AdditivePhylogeny

https://rosalind.info/problems/ba7c

In [7]:
from collections import defaultdict

def limb_length(D, j):
    n = len(D)
    limb = float('inf')
    for i in range(n):
        if i == j:
            continue
        for k in range(n):
            if k == j or k == i:
                continue
            val = (D[i][j] + D[j][k] - D[i][k]) / 2
            if val < limb:
                limb = val
    return limb

def find_attachment(D, n):
    """Find i,k such that Di,k = Di,n + Dn,k, return (i,k,Di,n)"""
    for i in range(n):
        if i == n:
            continue
        for k in range(n):
            if k == n or k == i:
                continue
            if abs(D[i][k] - (D[i][n] + D[n][k])) < 1e-6:
                return i, k, D[i][n]
    return None

def add_edge(tree, u, v, w):
    tree[u].append((v, w))
    tree[v].append((u, w))

def find_path(tree, start, end, path=None, visited=None):
    """Find path from start to end in tree (list of nodes)"""
    if path is None:
        path = [start]
    if visited is None:
        visited = set()
    visited.add(start)
    if start == end:
        return path
    for neighbor, _ in tree[start]:
        if neighbor not in visited:
            res = find_path(tree, neighbor, end, path + [neighbor], visited)
            if res:
                return res
    return None

def insert_node(tree, i, k, x, next_node_label):
    """Find point v at distance x from i on path i-k, insert internal node if necessary"""
    path = find_path(tree, i, k)
    d = 0
    for idx in range(len(path)-1):
        u = path[idx]
        v = path[idx+1]
        # find edge weight
        for neighbor, w in tree[u]:
            if neighbor == v:
                edge_w = w
                break
        if d + edge_w > x:
            # insert internal node
            new_node = next_node_label
            next_node_label += 1
            # adjust edges
            tree[u].remove((v, edge_w))
            tree[v].remove((u, edge_w))
            tree[u].append((new_node, x - d))
            tree[new_node].append((u, x - d))
            tree[new_node].append((v, edge_w - (x - d)))
            tree[v].append((new_node, edge_w - (x - d)))
            return new_node, next_node_label
        elif d + edge_w == x:
            return v, next_node_label
        d += edge_w
    return None, next_node_label

def additive_phylogeny(D, n, next_node_label=None):
    if next_node_label is None:
        next_node_label = n
    if n == 2:
        tree = defaultdict(list)
        add_edge(tree, 0, 1, D[0][1])
        return tree, next_node_label
    limb = limb_length(D, n-1)
    # subtract limb length from last column/row
    for j in range(n-1):
        D[j][n-1] -= limb
        D[n-1][j] = D[j][n-1]
    # find i,k such that Di,k = Di,n + Dn,k
    i, k, x = find_attachment(D, n-1)
    # remove last row/col
    D_trim = [row[:-1] for row in D[:-1]]
    tree, next_node_label = additive_phylogeny(D_trim, n-1, next_node_label)
    # insert internal node
    v, next_node_label = insert_node(tree, i, k, x, next_node_label)
    # add limb
    add_edge(tree, v, n-1, limb)
    return tree, next_node_label


In [9]:
# 读取文件
with open("data/rosalind_ba7c.txt", "r") as f:
    lines = [line.strip() for line in f if line.strip()]

n = int(lines[0])
D = [list(map(int, line.split())) for line in lines[1:]]

# 构建树
tree, _ = additive_phylogeny(D, n)

# 输出邻接表
for u in tree:
    for v, w in tree[u]:
        print(f"{u}->{v}:{int(w)}")


0->25:512
1->26:612
24->2:649
24->26:788
24->31:383
2->24:649
25->0:512
25->28:551
25->36:770
3->45:325
26->24:788
26->1:612
26->30:822
4->32:85
27->5:619
27->28:692
27->29:880
5->27:619
28->25:551
28->27:692
28->6:371
6->28:371
29->27:880
29->7:733
29->41:96
7->29:733
30->26:822
30->8:145
30->32:311
8->30:145
31->24:383
31->9:876
31->39:828
9->31:876
32->30:311
32->4:85
32->33:96
10->34:174
33->32:96
33->11:846
33->42:267
11->33:846
34->10:174
34->35:538
34->42:493
12->35:595
35->34:538
35->12:595
35->43:425
13->43:461
36->25:770
36->14:737
36->38:898
14->36:737
37->15:857
37->40:624
37->41:811
15->37:857
38->36:898
38->16:384
38->39:566
16->38:384
39->38:566
39->31:828
39->17:837
17->39:837
40->37:624
40->18:107
40->44:979
18->40:107
41->29:96
41->37:811
41->19:82
19->41:82
42->33:267
42->34:493
42->20:260
20->42:260
43->35:425
43->13:461
43->21:317
21->43:317
44->40:979
44->22:458
44->45:474
22->44:458
45->44:474
45->3:325
45->23:919
23->45:919
